<a href="https://colab.research.google.com/github/nferrucho/NPL/blob/main/curso3/ciclo3/1_extraccion_caracteristicas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://drive.google.com/uc?export=view&id=14reVO1X6LsjqJ3cFgoeHxxddZVGfZn3t" width="100%">

# Extracción de Características
---

En este notebook veremos una aproximación práctica al enfoque de ingeniería de características desde _Python_. Veremos algunas herramientas que podemos utilizar para extraer características de distintos tipos de datos.

Comenzamos instalando e importando las librerías necesarias:

In [ ]:
!apt install tree
!pip install dvc

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

## **1. Motivación**
---

La extracción de características en machine learning consiste en la transformación de datos crudos en representaciones numéricas más informativas y útiles para un algoritmo de aprendizaje automático. Esto se hace con el objetivo de mejorar la capacidad de un modelo para realizar una tarea determinada, como la clasificación o la regresión. La elección de las características adecuadas y su representación son cruciales para el éxito del modelo y pueden tener un impacto significativo en su precisión y rendimiento. Además, la extracción de características también puede ser útil para reducir la dimensionalidad de los datos y para eliminar características redundantes o irrelevantes.

<img src="https://drive.google.com/uc?export=view&id=1BtnrvfFspLnD1OE0STiTVwEwpEp9hCFW" width="80%">

La extracción de características se realiza por varias razones, incluyendo:

- **Mejora del rendimiento**: Al convertir los datos en una representación más informativa y relevante, se puede mejorar la capacidad del modelo para realizar una tarea determinada, lo que resulta en una mejora del rendimiento.
- **Reducción de la dimensionalidad**: La extracción de características también puede ser útil para reducir la dimensionalidad de los datos, lo que puede mejorar el tiempo de entrenamiento y prevenir el sobreajuste.
- **Eliminación de características irrelevantes o redundantes**: La extracción de características puede ayudar a eliminar características irrelevantes o redundantes, lo que puede mejorar la interpretabilidad y la eficiencia del modelo.
- **Facilitación de la interpretabilidad**: Al convertir los datos en una representación más informativa y relevante, también se puede facilitar la interpretabilidad del modelo y su capacidad para hacer inferencias.

La extracción de características varía en dependencia del tipo de datos que estemos manejando. Veamos un ejemplo con distintas herramientas especializadas y `dvc`, comenzamos creando una carpeta donde tendremos el repositorio:

In [ ]:
!mkdir features
%cd features

Ahora configuramos `git`:

In [ ]:
!git config --global user.email "ejemplo@unal.edu.co"
!git config --global user.name "Ejemplo"
!git config --global init.defaultBranch master

Inicializamos el repositorio:

In [ ]:
!git init

Ahora, vamos a crear un folder para almacenar datos crudos y modelos.

In [ ]:
!mkdir -p data/raw data/features

Veamos la estructura de directorios que tenemos:

In [ ]:
!tree

Inicializamos `dvc`:

In [ ]:
!dvc init

Creamos un commit con `dvc`:

In [ ]:
!git add .dvc
!git commit -m "Inicializamos dvc"

## **2. Imágenes**
---

La extracción de características en imágenes es un proceso en el aprendizaje automático y la visión por computadora donde se extraen y se representan características relevantes de imágenes para su posterior uso en tareas como clasificación, detección, segmentación, etc. Estas características pueden ser formas, texturas, colores, entre otros; su representación suele ser en forma de vectores numéricos. La elección de las características a extraer y cómo representarlas es importante para el rendimiento de la tarea.

Vamos a descargar el conjunto de datos Olivetti Faces, el cual es un conjunto de imágenes de rostros humanos comúnmente utilizado en investigaciones en el campo de la visión por computadora y el aprendizaje automático. Contiene imágenes de 40 personas diferentes, cada una de ellas tomada en 4 posiciones diferentes y en 64x64 píxeles. Cada imagen está representada en escala de grises y está normalizada para tener una iluminación uniforme.

Este conjunto de datos se utiliza comúnmente para evaluar técnicas de extracción de características faciales y para comparar diferentes algoritmos de clasificación. Es un conjunto de datos pequeño y fácilmente accesible, por lo que es ideal para experimentos iniciales en el campo. Sin embargo, debido a su tamaño limitado y a la simplicidad de las imágenes, también se considera que es limitado en términos de complejidad y diversidad de las caras representadas.

Vamos a cargarlo dentro de la carpeta de datos crudos:

In [ ]:
!wget https://raw.githubusercontent.com/mindlab-unal/mlds6-datasets/main/u3/olivetti.npy -O data/raw/olivetti.npy

Podemos validar el estado del repositorio:

In [ ]:
!git status

Agregamos los datos con `dvc`:

In [ ]:
!dvc add data/raw/olivetti.npy

Ahora, vamos a agregar los metadatos creados por `dvc`:

In [ ]:
!git add data/raw/olivetti.npy.dvc data/raw/.gitignore

Creamos un commit con estos datos

In [ ]:
!git commit -m "Agregamos el conjunto de datos Olivetti"

Ahora, cargamos el conjunto de datos para inspeccionarlo:

In [ ]:
data = np.load("data/raw/olivetti.npy")
display(data.shape)

Los datos son un arreglo de `numpy` que contiene 400 imágenes de tamaño 64x64.

Veamos una imagen aleatoria del conjunto de datos:

In [ ]:
idx = np.random.randint(data.shape[0])
img = data[idx]

Visualizamos la imagen:

In [ ]:
fig, ax = plt.subplots()
ax.imshow(img, cmap="gray")
ax.axis("off")
fig.show()

Ahora, veamos cómo podemos convertir las imágenes en un vector de características. Para esto usaremos la librería `scikit-image`. Se trata de una biblioteca de software libre para procesamiento de imágenes en _Python_. Se enfoca en proporcionar herramientas eficientes y fáciles de usar para realizar tareas comunes de procesamiento de imágenes.

En este caso utilizaremos una técnica clásica de procesamiento de imágenes conocida como HOG, o *Histogram of Oriented Gradients*. Se trata de una técnica de extracción de características utilizada para describir regiones de una imagen en términos de la distribución de gradientes de intensidad de la luz en diferentes orientaciones, como se muestra en la siguiente figura:

<img src="https://drive.google.com/uc?export=view&id=13Tcijgc2hcoIl5DNo9dsj01FVpKxK9yj" width="80%">

En resumen, la técnica HOG consiste en los siguientes pasos:

Cálculo de gradientes: Se calcula el gradiente de intensidad de la luz en diferentes direcciones en cada punto de la imagen.

1. **Discretización**: Se discretiza el espacio de orientación en un número finito de direcciones.
2. **Agrupación en celdas**: Se divide la imagen en celdas pequeñas y se agrupan los gradientes dentro de cada celda.
3. **Cálculo de histogramas**: Se calcula un histograma de orientación para cada celda, describiendo la distribución de gradientes dentro de ella.
4. **Concatenación de histogramas**: Se concatenan los histogramas de orientación de todas las celdas para formar un vector de características.

Este vector de características se utiliza como entrada para un algoritmo de aprendizaje automático, como un clasificador, para realizar tareas como la detección de objetos o la identificación de características. La técnica HOG es ampliamente utilizada en el campo de la visión por computadora debido a su capacidad para describir la distribución de características en una imagen de manera efectiva y compacta.

Comenzamos instalando `scikit-image`:

In [ ]:
!pip install scikit-image

Importamos la función `hog` para aplicarlo sobre las imágenes:

In [ ]:
from skimage.feature import hog

Veamos las características para la imagen que teníamos seleccionada, usamos algunos hiperparámetros como:

- `orientations`: direcciones de gradiente a considerar.
- `pixels_per_cell`: tamaño de las celdas donde se calculan los gradientes.

In [ ]:
features = hog(img, orientations=4, pixels_per_cell=(16, 16))
display(features.shape)

In [ ]:
features

Como podemos ver, obtenemos un vector de tamaño `144`. Podemos repetir este proceso para todas las imágenes para obtener una matriz de características de todo el dataset:

In [ ]:
features = np.concatenate([
    hog(data[i], orientations=4, pixels_per_cell=(16, 16))[np.newaxis, ...]
    for i in range(data.shape[0])
    ])
display(features.shape)

Guardamos las características dentro del repositorio:

In [ ]:
np.save("data/features/olivetti.npy", features)

Agregamos las características a `dvc`:

In [ ]:
!dvc add data/features/olivetti.npy
!git add data/features/olivetti.npy.dvc data/features/.gitignore

Creamos un commit con las características extraídas de las imágenes:

In [ ]:
!git commit -m "Agregamos características de olivetti"

## **2. Texto**
---

La extracción de características en textos es el proceso de convertir un corpus de texto en una representación numérica que pueda ser utilizada por un algoritmo de aprendizaje automático. Se trata de un paso crítico en la mayoría de los sistemas de procesamiento de lenguaje natural (NLP, por sus siglas en inglés) y de análisis de texto, ya que permite a los algoritmos trabajar con los datos en una forma que puedan comprender y utilizar.

Hay muchas técnicas diferentes de extracción de características para textos, dependiendo del problema que se esté tratando de resolver y del tipo de corpus que se esté utilizando. Algunas técnicas comunes incluyen:

- **Bolsa de palabras**: Se representa cada documento como una lista de frecuencias de palabras, ignorando el orden y la gramática de las palabras.
- **N-gramas**: Se representa cada documento como una lista de frecuencias de secuencias de n palabras.
- **Representación distribucional**: Se representa cada documento como un vector que describe las probabilidades de co-ocurrencia de las palabras en el corpus.
- **Modelos semánticos**: Se representa cada documento como un vector que describe la semántica subyacente, utilizando técnicas como word2vec o GloVe.

> **Nota**: puede ver más detalle de estos modelos en el módulo 4 de procesamiento y entendimiento de lenguaje natural.

La extracción de características es una tarea importante y desafiante en el NLP, ya que debe capturar de manera efectiva la información relevante y suprimir la información irrelevante o redundante en el corpus. La elección de la técnica adecuada de extracción de características puede tener un impacto significativo en la efectividad de los sistemas de NLP y de análisis de texto.

En este caso utilizaremos un modelo semántico contenido en la librería `spacy`, la cual es una biblioteca de procesamiento de lenguaje natural de código abierto para _Python_. Se destaca por su alta velocidad y eficiencia, lo que la convierte en una opción popular para muchos problemas de NLP.

<img src="https://drive.google.com/uc?export=view&id=1nk8ZGj27YhT-GCZtlyJq1c5P_AuPoFLr" width="80%">

Además, spaCy ofrece una amplia gama de modelos pre-entrenados para muchos idiomas, incluyendo inglés, alemán, francés, español, italiano, portugués, holandés, danés, sueco, finlandés y noruego, lo que permite a los usuarios utilizar sus funciones de forma rápida y efectiva sin tener que entrenar desde cero.

Comenzamos instalando `spacy`:

In [ ]:
!pip install spacy

Importamos `spacy`:

In [ ]:
import spacy

Vamos a descargar un _Pipeline_ de `spacy` para su uso en el idioma inglés:

In [ ]:
spacy.cli.download("en_core_web_sm")

Cargamos el _Pipeline_:

In [ ]:
nlp = spacy.load("en_core_web_sm")

Vamos a cargar el conjunto de datos [Twitter Financial News](https://www.kaggle.com/datasets/sulphatet/twitter-financial-news) de Kaggle. El cual contiene textos de tweets relacionados con finanzas.

Cargamos el conjunto de datos:

In [ ]:
!wget https://raw.githubusercontent.com/mindlab-unal/mlds6-datasets/main/u3/twitter_financial.parquet -O data/raw/twitter_financial.parquet

Agregamos los datos con `dvc` y creamos el commit correspondiente.

In [ ]:
!dvc add data/raw/twitter_financial.parquet
!git add data/raw/.gitignore data/raw/twitter_financial.parquet.dvc
!git commit -m "Agregamos los datos de twitter financial"

Ahora veamos el proceso de extracción de características sobre texto. Primero cargamos el conjunto de datos:

In [ ]:
data = pd.read_parquet("data/raw/twitter_financial.parquet")
display(data.head())

Veamos cómo podemos extraer características con el _Pipeline_, lo haremos con un documento aleatorio del conjunto de datos:

In [ ]:
doc = data.loc[np.random.randint(data.shape[0]), "text"]
display(doc)

Extraemos un vector de características:

In [ ]:
features = nlp(doc).vector
display(features.shape)

In [ ]:
features

Como podemos ver, obtenemos un vector de características de tamaño 96.

Ahora representamos todo el conjunto de datos:

In [ ]:
features = np.concatenate([
    nlp(doc).vector[np.newaxis, ...]
    for doc in data.text.tolist()
    ])
display(features.shape)

Exportamos las características:

In [ ]:
np.save("data/features/twitter_financial.npy", features)

Por último, agregamos las características a `dvc` y creamos un commit:

In [ ]:
!dvc add data/features/twitter_financial.npy
!git add data/features/.gitignore data/features/twitter_financial.npy.dvc
!git commit -m "Agregamos las características de twitter financial"

## **3. Series de Tiempo**
---

La extracción de características con series de tiempo es el proceso de transformar una serie temporal en una representación numérica que pueda ser utilizada por un algoritmo de aprendizaje automático. Esto es necesario debido a que los algoritmos de aprendizaje automático no pueden trabajar directamente con datos temporales, ya que los datos temporales son secuencias de valores que varían con el tiempo.

<img src="https://drive.google.com/uc?export=view&id=1JyzLpm5nfSdY31p3xw5zIG3v2vARuh4e" width="80%">

Hay muchas técnicas diferentes para la extracción de características con series de tiempo. La elección de la técnica adecuada depende de la naturaleza de la serie temporal y del problema que se esté tratando de resolver. Algunas técnicas comunes incluyen:

- **Transformadas**: Se utiliza una transformada matemática, como la transformada de Fourier, para analizar la frecuencia y el espectro de la serie temporal.
- **Características estadísticas**: Se extraen características estadísticas como la media, la desviación estándar, la mediana y la moda de la serie temporal.
- **Características de la tendencia**: Se extraen características que describen la tendencia de la serie temporal, como la tasa de crecimiento o la tasa de cambio.
- **Características de la estacionalidad**: Se extraen características que describen la estacionalidad de la serie temporal, como la frecuencia y la amplitud de los patrones estacionales.

La extracción de características es un paso crítico en el análisis de series de tiempo, ya que permite a los algoritmos de aprendizaje automático trabajar con los datos en una forma que puedan comprender y utilizar. Además, es importante seleccionar cuidadosamente las características a extraer para evitar la inclusión de características irrelevantes o redundantes, que pueden perjudicar la efectividad de los algoritmos.

En este caso utilizaremos la librería `tsfresh` para extracción de características a partir de series de tiempo. Se trata de una biblioteca de _Python_ que se enfoca en la extracción de características de series de tiempo. Es una herramienta automatizada para la selección y la extracción de características que son relevantes y útiles para el análisis de series de tiempo.

tsfresh utiliza un enfoque de aprendizaje automático para identificar las características relevantes, además utiliza una variedad de técnicas estadísticas y matemáticas para extraer esas características de los datos. Estas características se pueden utilizar en una variedad de aplicaciones, incluyendo la clasificación, la regresión y la detección de anomalías.

tsfresh es una herramienta muy útil para los investigadores y los científicos de datos que trabajan con series de tiempo, ya que permite extraer características relevantes de los datos de forma rápida y efectiva, sin tener que escribir código para realizar esta tarea manualmente. Además, tsfresh es fácil de usar y se integra con otras bibliotecas de aprendizaje automático y análisis de datos de _Python_.

Veamos cómo es su instalación:

In [ ]:
!pip install tsfresh

Vamos a descargar el conjunto de datos [Robot Execution Failures](http://archive.ics.uci.edu/ml/datasets/Robot+Execution+Failures) que contiene mediciones de fuerza y torque de fallos de un robot en el tiempo.

In [ ]:
!wget https://raw.githubusercontent.com/mindlab-unal/mlds6-datasets/main/u3/robot_failures.parquet -O data/raw/robot_failures.parquet

Ahora, agregamos los datos a `dvc` y `git`:

In [ ]:
!dvc add data/raw/robot_failures.parquet
!git add data/raw/.gitignore data/raw/robot_failures.parquet.dvc
!git commit -m "Agregamos los datos de robot failures"

Cargamos los datos con `pandas`:

In [ ]:
data = pd.read_parquet("data/raw/robot_failures.parquet")
display(data.head())

El conjunto de datos contiene las siguientes columnas:

- `id`: identificador de la serie de tiempo.
- `time`: tiempo.
- `F_i`: fuerza ejercida en un eje `i`.
- `T_i`: torque ejercido en un eje `i`.

Veamos una gráfica de una serie de tiempo:

In [ ]:
fig, ax = plt.subplots()
data.query("id == 20").plot(x="time", subplots=True, sharex=True, ax=ax)
fig.show()

Para extraer características con `tsfresh` podemos usar la función `extract_features`:

In [ ]:
from tsfresh import extract_features

Extraemos las características, para ello usamos los parámetros:

- `column_id`: específica el identificador de cada serie de tiempo.
- `column_sort`: específica el tiempo.

In [ ]:
features = extract_features(data, column_id="id", column_sort="time")
display(features.shape)

Obtenemos 4698 características (de los tipos que mencionamos anteriormente) para cada una de las 88 series de tiempo en el conjunto de datos.

Vamos a exportar estas características:

In [ ]:
features.to_parquet("data/features/robot_failures.parquet")

Agregamos las características al repositorio con `dvc`:

In [ ]:
!dvc add data/features/robot_failures.parquet
!git add data/features/.gitignore data/features/robot_failures.parquet.dvc
!git commit -m "Agregamos las características de robot failures"

## **4. Aprendizaje de Características**
---

El aprendizaje de características es un proceso en el que se utiliza un algoritmo de machine learning para automáticamente identificar y extraer características relevantes de los datos de entrada. La idea detrás de esto es que la mayoría de los algoritmos de machine learning funcionan mejor con datos en formato numérico, y los datos originales (por ejemplo, imágenes, audio, texto, etc.) no siempre están en este formato.

<img src="https://drive.google.com/uc?export=view&id=1O_nI1j8R_c1Zjher9rpl_w9MhvdFJIyr" width="80%">

El proceso de aprendizaje de características se divide en dos partes: la extracción de características y la selección de características. La extracción de características se refiere a la conversión de los datos originales en una forma que se pueda utilizar en un algoritmo de machine learning. La selección de características se refiere a la identificación de las características más relevantes y significativas que se deben utilizar en el algoritmo de aprendizaje.

El aprendizaje de características es un paso crítico en muchos problemas de machine learning, ya que ayuda a mejorar la precisión y el rendimiento de los algoritmos. Por ejemplo, si un algoritmo de aprendizaje de máquina recibe como entrada características irrelevantes o no significativas, su precisión disminuirá y tendrá un rendimiento más bajo. Por lo tanto, es importante realizar una buena extracción y selección de características para obtener buenos resultados en el aprendizaje de máquinas.

Vamos a ver un modelo de autoencoder para aprendizaje de características con `tensorflow`. Un autoencoder es un tipo de red neuronal artificial que se utiliza para aprender una representación compacta y densa de los datos de entrada. Se compone de dos partes principales: un codificador que convierte los datos de entrada en una representación compacta, y un decodificador que intenta reconstruir los datos de entrada originales a partir de la representación comprimida.

El objetivo de un autoencoder es aprender una representación de los datos que sea lo suficientemente compacta para que se pueda usar como entrada en otras tareas, como la clasificación o la detección de anormalidades. Durante el entrenamiento, el autoencoder recibe los datos de entrada y compara la reconstrucción realizada por el decodificador con los datos de entrada originales, y se ajusta para minimizar la diferencia entre las dos.

Comenzamos importando los componentes de `tensorflow` que necesitamos:

In [ ]:
from keras.models import Sequential
from keras.layers import Input, Dense
from keras.optimizers import Adam
from keras.losses import MeanSquaredError

Vamos a reutilizar el conjunto de datos de Olivetti:

In [ ]:
data = np.load("data/raw/olivetti.npy")
print(data.shape)

Vamos a extraer todos los píxeles de las imágenes como vectores individuales:

In [ ]:
data = data.reshape(400, -1)
print(data.shape)

Ahora vamos a implementar un modelo de Autoencoder, comenzamos con el codificador:

In [ ]:
encoder = Sequential([
    Input(shape=(4096, )),
    Dense(units=128, activation="relu"),
    Dense(units=64, activation="linear")
    ])

Ahora el decodificador:

In [ ]:
decoder = Sequential([
    Input(shape=(64, )),
    Dense(units=128, activation="relu"),
    Dense(units=4096, activation="linear"),
    ])

Implementamos el modelo completo:

In [ ]:
autoencoder = Sequential([
    Input(shape=(4096, )),
    encoder,
    decoder
    ])
autoencoder.summary()

Compilamos el modelo:

In [ ]:
autoencoder.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss=MeanSquaredError()
    )

Ahora lo entrenamos:

In [ ]:
history = autoencoder.fit(data, data, epochs=100, batch_size=256)

Veamos la pérdida del modelo:

In [ ]:
fig, ax = plt.subplots()
ax.plot(history.history["loss"])
ax.set_xlabel("Épocas")
ax.set_ylabel("Pérdida")
fig.show()

Podemos utilizar el modelo para extraer características de las imágenes de la siguiente forma:

In [ ]:
features = encoder.predict(data)
display(features.shape)

Como podemos ver, obtenemos una representación de tamaño 64 para cada una de las 400 imágenes. Se trata de una representación neuronal que probablemente no se interprete tan fácilmente. No obstante, es útil si deseamos entrenar algún modelo a partir de estas imágenes, además resulta ser más específica que representaciones clásicas como HOG.

Exportamos las características:

In [ ]:
np.save("data/features/learned.npy", features)

Finalmente, las agregamos al repositorio:

In [ ]:
!dvc add data/features/learned.npy
!git add data/features/.gitignore data/features/learned.npy.dvc
!git commit -m "Agregamos las características aprendidas"

Como pudimos ver, existen distintas alternativas para extracción de características en dependencia del tipo de dato. De hecho, existen muchas técnicas específicas para cada tipo de dato o aplicación. Por ejemplo, características en imágenes médicas, en imágenes térmicas, en detección de rostros, entre otras.

No obstante, es importante entender el proceso de extracción de características —independiente de la librería o los datos que usemos— y su integración con herramientas de versionamiento como `git` y `dvc`.

## Créditos
---

**Profesor**

- [Jorge E. Camargo, PhD](https://dis.unal.edu.co/~jecamargom/)

**Asistente docente**:

- [Juan S. Lara MSc](https://www.linkedin.com/in/juan-sebastian-lara-ramirez-43570a214/)

**Diseño de imágenes:**
- [Brian Chaparro Cetina](mailto:bchaparro@unal.edu.co).

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*